In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
import tensorflow as tf
from termcolor import colored

In [2]:
# Checking if GPU available
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
chars_for_add = '0123456789+'
num_features = len(chars_for_add)
char_to_index = dict((c, i) for i, c in enumerate(chars_for_add))
index_to_char = dict((i, c) for i, c in enumerate(chars_for_add))

In [21]:
def generate_samples_for_add():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    sample = str(first) + '+' + str(second)
    label = str(first+second)
    return sample, label
generate_samples_for_add()

('74+84', '158')

In [22]:
hidden_units = 128
max_time_steps = 7


    # Simple RNN
"""Fully-connected RNN where output is to be fed back to input.
    units: Positive integer, dimensionality of the output space.
    activation: Activation function to use. Default: hyperbolic 
    tangent (tanh). If you pass None, no activation is applied (ie.
    "linear" activation: a(x) = x).
    return_sequences: Boolean. Whether to return the last output in
    the output sequence, or the full sequence. Default: False."""

    # Repeat Vector
"""Repeats the input max_time_steps times"""

    # Time distributed layer
"""This wrapper allows to apply a layer to every temporal slice of
    an input.
    The input should be at least 3D, and the dimension of index one 
    will be considered to be the temporal dimension."""

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    
    RepeatVector(max_time_steps),
    
    SimpleRNN(hidden_units, return_sequences=True),
    
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 7, 128)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 7, 128)            32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [23]:
def vectorize_sample(sample, label):
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    diff_x = max_time_steps - len(sample)
    diff_y = max_time_steps - len(label)
    for i, c in enumerate(sample):
        x[i+diff_x, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[i+diff_y, char_to_index[c]] = 1
    for i in range(diff_x):
        y[i, char_to_index['0']] = 1
    return x,y    
        
e, l = generate_samples_for_add()
print(e, l)
x, y = vectorize_sample(e, l)
print(x.shape, y.shape)

6+30 36
(7, 11) (7, 11)


In [24]:
def devectorize_sample(sample):
    res = [index_to_char[np.argmax(vec)] for i, vec in enumerate(sample)]
    return ''.join(res)

devectorize_sample(x)

'0006+30'

In [25]:
devectorize_sample(y)

'0000036'

In [26]:
# Dataset
def create_dataset(num_samples=5000):
    x = np.zeros((num_samples, max_time_steps, num_features))
    y = np.zeros((num_samples, max_time_steps, num_features))
    for i in range(num_samples):
        e, l = generate_samples_for_add()
        e, l = vectorize_sample(e, l)
        x[i] = e
        y[i] = l
    return x, y    

x, y = create_dataset()
print(x.shape, y.shape)
devectorize_sample(x[78])


(5000, 7, 11) (5000, 7, 11)


'0090+65'

In [27]:
devectorize_sample(y[78])


'0000155'

In [28]:
# Model Training
# Callbacks: utilities called at certain points during model training.
# on_epoch_begin	called at the beginning of every epoch.
# on_epoch_end	called at the end of every epoch.
l_cb = LambdaCallback(
    on_epoch_end = lambda e, l:print('{:.2f}'.format(l['val_accuracy']), end=' _ ')
)

# EarlyStopping: Stop training when a monitored metric has stopped improving.
# monitor	Quantity to be monitored.
# patience	Number of epochs with no improvement after which training will be stopped.
es_cb = EarlyStopping(monitor='val_loss', patience=20)


# verbose	0 or 1. Verbosity mode. 0 = silent, 1 = progress bar.
model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2,
         verbose=True , callbacks=[es_cb, l_cb])



Epoch 1/500
16/16 [==============================] - 0s 25ms/step - loss: 0.9468 - accuracy: 0.5925 - val_loss: 0.7093 - val_accuracy: 0.5443
Epoch 2/500
16/16 [==============================] - 0s 14ms/step - loss: 0.6809 - accuracy: 0.5456 - val_loss: 0.6709 - val_accuracy: 0.5453
Epoch 3/500
16/16 [==============================] - 0s 15ms/step - loss: 0.6622 - accuracy: 0.5556 - val_loss: 0.6645 - val_accuracy: 0.5839
Epoch 4/500
16/16 [==============================] - 0s 14ms/step - loss: 0.6600 - accuracy: 0.5972 - val_loss: 0.6636 - val_accuracy: 0.6140
Epoch 5/500
16/16 [==============================] - 0s 14ms/step - loss: 0.6565 - accuracy: 0.5761 - val_loss: 0.6610 - val_accuracy: 0.6576
Epoch 6/500
16/16 [==============================] - 0s 14ms/step - loss: 0.6551 - accuracy: 0.6430 - val_loss: 0.6597 - val_accuracy: 0.6269
Epoch 7/500
16/16 [==============================] - 0s 16ms/step - loss: 0.6525 - accuracy: 0.6254 - val_loss: 0.6580 - val_accuracy: 0.6161
Epoch 

16/16 [==============================] - 0s 16ms/step - loss: 0.0375 - accuracy: 0.8286 - val_loss: 0.0602 - val_accuracy: 0.8150
Epoch 59/500
16/16 [==============================] - 0s 13ms/step - loss: 0.0292 - accuracy: 0.8305 - val_loss: 0.0497 - val_accuracy: 0.8189
Epoch 60/500
16/16 [==============================] - 0s 14ms/step - loss: 0.0235 - accuracy: 0.8317 - val_loss: 0.0457 - val_accuracy: 0.8214
Epoch 61/500
16/16 [==============================] - 0s 16ms/step - loss: 0.0210 - accuracy: 0.8323 - val_loss: 0.0445 - val_accuracy: 0.8194
Epoch 62/500
16/16 [==============================] - 0s 16ms/step - loss: 0.0193 - accuracy: 0.8324 - val_loss: 0.0439 - val_accuracy: 0.8200
Epoch 63/500
16/16 [==============================] - 0s 14ms/step - loss: 0.0184 - accuracy: 0.8317 - val_loss: 0.0422 - val_accuracy: 0.8187
Epoch 64/500
16/16 [==============================] - 0s 16ms/step - loss: 0.0176 - accuracy: 0.8309 - val_loss: 0.0421 - val_accuracy: 0.8204
Epoch 65/500

16/16 [==============================] - 0s 15ms/step - loss: 0.0026 - accuracy: 0.8298 - val_loss: 0.0233 - val_accuracy: 0.8217
Epoch 116/500
16/16 [==============================] - 0s 17ms/step - loss: 0.0025 - accuracy: 0.8294 - val_loss: 0.0227 - val_accuracy: 0.8233
Epoch 117/500
16/16 [==============================] - 0s 16ms/step - loss: 0.0024 - accuracy: 0.8292 - val_loss: 0.0234 - val_accuracy: 0.8249
Epoch 118/500
16/16 [==============================] - 0s 14ms/step - loss: 0.0024 - accuracy: 0.8298 - val_loss: 0.0234 - val_accuracy: 0.8236
Epoch 119/500
16/16 [==============================] - 0s 15ms/step - loss: 0.0023 - accuracy: 0.8291 - val_loss: 0.0230 - val_accuracy: 0.8240
Epoch 120/500
16/16 [==============================] - 0s 19ms/step - loss: 0.0023 - accuracy: 0.8294 - val_loss: 0.0240 - val_accuracy: 0.8214
Epoch 121/500
16/16 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.8296 - val_loss: 0.0227 - val_accuracy: 0.8219
Epoch 

16/16 [==============================] - 0s 17ms/step - loss: 8.2733e-04 - accuracy: 0.8294 - val_loss: 0.0205 - val_accuracy: 0.8226
Epoch 172/500
16/16 [==============================] - 0s 18ms/step - loss: 8.2419e-04 - accuracy: 0.8293 - val_loss: 0.0211 - val_accuracy: 0.8229
Epoch 173/500
16/16 [==============================] - 0s 20ms/step - loss: 8.1323e-04 - accuracy: 0.8291 - val_loss: 0.0205 - val_accuracy: 0.8224


In [29]:
x_test, y_test = create_dataset(10)
preds = model.predict(x_test)

for i, pred in enumerate(preds):
    y = devectorize_sample(y_test[i])
    y_pred = devectorize_sample(pred)
    col = 'green'
    if y!=y_pred:
        col='red'
    out = 'Input: '+devectorize_sample(x_test[i])+ ' Out:'+y+' Pred: '+y_pred
    print(colored(out, col))

Input: 0035+20 Out:0000055 Pred: 0000155
Input: 00025+9 Out:0000034 Pred: 0000134
Input: 0038+37 Out:0000075 Pred: 0002175
Input: 0001+37 Out:0000038 Pred: 0000238
Input: 0013+85 Out:0000098 Pred: 0001198
Input: 0028+19 Out:0000047 Pred: 0001247
Input: 0070+47 Out:0000117 Pred: 0002117
Input: 0004+96 Out:0000100 Pred: 0003100
Input: 0047+11 Out:0000058 Pred: 0000458
Input: 0051+79 Out:0000130 Pred: 0001130
